In [2]:
pip install simfin

Note: you may need to restart the kernel to use updated packages.


In [3]:
api_key = '9104e7ab-b221-4476-89fe-0f3e0300ad24'

In [4]:
import simfin as sf
from simfin.names import *

# Set your API-key for downloading data.
# Replace YOUR_API_KEY with your actual API-key.
sf.set_api_key(api_key)

# Set the local directory where data-files are stored.
# The dir will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Load the annual Income Statements for all companies in the US.
# The data is automatically downloaded if you don't have it already.
df = sf.load_income(variant='annual', market='us')

# Print all Revenue and Net Income for Microsoft (ticker MSFT).
print(df.loc['MSFT', [REVENUE, NET_INCOME]])

Dataset "us-income-annual" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
                  Revenue   Net Income
Report Date                           
2019-06-30   1.258430e+11  39240000000
2020-06-30   1.430150e+11  44281000000
2021-06-30   1.680880e+11  61271000000
2022-06-30   1.982700e+11  72738000000
2023-06-30   2.119150e+11  72361000000


/opt/anaconda3/lib/python3.12/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load datasets with improved error handling
companies_df = pd.read_csv('us-companies.csv', sep=None, engine='python', on_bad_lines='skip', encoding='utf-8')
share_prices_df = pd.read_csv('us-shareprices-daily.csv', sep=None, engine='python', on_bad_lines='skip', encoding='utf-8')

# Strip column names of any whitespace
share_prices_df.columns = share_prices_df.columns.str.strip()
companies_df.columns = companies_df.columns.str.strip()

# Print column names for debugging
print("Columns in share_prices_df:", share_prices_df.columns.tolist())

# Ensure 'ticker' column exists before filtering
if 'Ticker' in share_prices_df.columns:
    apple_data = share_prices_df[share_prices_df['Ticker'] == 'AAPL']
else:
    raise KeyError("The 'ticker' column is missing from share_prices_df.")

# Merge datasets on the common key (assuming 'ticker' is the identifier)
apple_data = apple_data.merge(companies_df, on='Ticker', how='left')

# Convert date column to datetime format
apple_data['date'] = pd.to_datetime(apple_data['Date'], errors='coerce')

# Drop rows with invalid dates
apple_data = apple_data.dropna(subset=['date'])

# Sort by date
apple_data.sort_values(by='date', inplace=True)

# Feature Engineering: Create lag features (previous day's price)
apple_data['prev_close'] = apple_data['Close'].shift(1)

# Moving Averages (e.g., 7-day and 30-day)
apple_data['ma7'] = apple_data['Close'].rolling(7).mean()
apple_data['ma30'] = apple_data['Close'].rolling(30).mean()

# Target Variable: Next day close price
apple_data['next_day_close'] = apple_data['Close'].shift(-1)

# Drop rows with NaN values introduced by shifting
cleaned_data = apple_data.dropna()

# Encode categorical variables (e.g., industry)
if 'industry' in cleaned_data.columns:
    le = LabelEncoder()
    cleaned_data['industry'] = le.fit_transform(cleaned_data['industry'].astype(str))

# Select relevant columns for modeling
features = ['prev_close', 'ma7', 'ma30', 'industry']
target = 'next_day_close'

X = cleaned_data[features]
y = cleaned_data[target]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train = pd.DataFrame(X_train_scaled, columns=features)
X_test = pd.DataFrame(X_test_scaled, columns=features)

# Save processed data
X_train.to_csv('/mnt/data/X_train.csv', index=False)
X_test.to_csv('/mnt/data/X_test.csv', index=False)
y_train.to_csv('/mnt/data/y_train.csv', index=False)
y_test.to_csv('/mnt/data/y_test.csv', index=False)

print("ETL process completed for Apple. Processed datasets saved.")


Columns in share_prices_df: ['Ticker', 'SimFinId', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding']


KeyError: "['industry'] not in index"